In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import lit

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
  .appName('Jupyter BigQuery Storage')\
  .config('spark.jars', 'gs://spark-lib/bigquery/spark-bigquery-latest_2.12.jar') \
  .getOrCreate()

In [ ]:
!hadoop distcp gs://bkt-prd-portfolio/raw/censo/2018/alunos/DM_ALUNO.CSV hdfs://cluster-censo-educacional-m:8020//raw/DM_ALUNOS.CSV
!hadoop distcp gs://bkt-prd-portfolio/raw/censo/2018/ies/DM_IES.CSV hdfs://cluster-censo-educacional-m:8020//raw/DM_IES.CSV
!hadoop distcp gs://bkt-prd-portfolio/raw/censo/2018/curso/DM_CURSO.CSV hdfs://cluster-censo-educacional-m:8020//raw/DM_CURSO.CSV
!hadoop distcp gs://bkt-prd-portfolio/raw/censo/2018/municipios/municipios.csv hdfs://cluster-censo-educacional-m:8020//raw/municipios.csv

In [ ]:
hdfs = "hdfs://cluster-censo-educacional-m:8020//"

In [ ]:
!hdfs dfs -ls hdfs://cluster-censo-educacional-m:8020//raw/

In [ ]:
def read_file_csv(file, charset="iso-8859-1"):
    return spark\
    .read\
    .option("charset", charset)\
    .format("csv")\
    .option("inferSchema", "true")\
    .option("header", "true")\
    .option("delimiter", "|")\
    .load(str(hdfs)+"raw/"+str(file))

In [ ]:
def read_file_parquet(file,charset="iso-8859-1"):
    return spark.read.option("charset", charset).parquet(hdfs+"stg/parquet/"+str(file)+"/*.parquet")

In [ ]:
def write_file_parquet(df,zone, filename):
    df.write.mode('overwrite').parquet(hdfs+str(zone)+"/parquet/"+str(filename)+"/")

In [ ]:
def write_bigquery(df, table):
    df.write.format('bigquery') \
    .option("mapreduce.fileoutputcommitter.algorithm.version", "2")\
    .option('table', 'dm_censo.'+str(table)) \
    .option("temporaryGcsBucket","bkt-prd-portfolio/temp") \
    .mode("append") \
    .save()


In [ ]:
def clean_data():
    ### TRATAMENTO DE DADOS MUNICIPIOS IBGE -- DIM_LOCALIZACAO
    spark.sql("""SELECT coalesce(sha2(cd_uf||NM_UF||CD_MUNICIPIO||CD_MUNICIPIO_COMPLETO||NM_MUNICIPIO, 256),'N/A') AS CD_HASH, 
                    sha2(CD_UF||CD_MUNICIPIO_COMPLETO,256) as HASH_LOCALIZACAO,
                    * FROM(
                    SELECT DISTINCT COALESCE(UF,-3) AS CD_UF, 
                    COALESCE(UPPER(NOME_UF),'N/A') AS NM_UF,
                    COALESCE(MUNICIPIO,-3) AS CD_MUNICIPIO,
                    COALESCE(CODIGO_MUNICIPIO_COMPLETO,-3) AS CD_MUNICIPIO_COMPLETO,
                    COALESCE(UPPER(NOME_MUNICIPIO),'N/A') AS NM_MUNICIPIO,
                    CURRENT_DATE AS DT_INSERT
            FROM stg_municipio_ibge) TB """).createOrReplaceTempView("DIM_LOCALIZACAO")
    
    ### TRATAMENTO DE DADOS DIM_IES
    spark.sql("""SELECT
                COALESCE(NU_ANO_CENSO,-3) AS NU_ANO_CENSO,
                COALESCE(CO_IES,-3) AS CO_IES,
                UPPER(COALESCE(NO_IES,'N/A')) AS NO_IES,
                UPPER(COALESCE(SG_IES,'N/A')) AS SG_IES,
                COALESCE(CO_MANTENEDORA,-3) AS CO_MANTENEDORA,
                UPPER(COALESCE(NO_MANTENEDORA,'N/A')) AS NO_MANTENEDORA,
                COALESCE(TP_CATEGORIA_ADMINISTRATIVA,-3) AS TP_CATEGORIA_ADMINISTRATIVA,
                CASE WHEN TP_CATEGORIA_ADMINISTRATIVA = 1 THEN 'PUBLICA FEDERAL'
                     WHEN TP_CATEGORIA_ADMINISTRATIVA = 2 THEN 'PUBLICA ESTADUAL'
                     WHEN TP_CATEGORIA_ADMINISTRATIVA = 3 THEN 'PUBLICA MUNICIPAL'
                     WHEN TP_CATEGORIA_ADMINISTRATIVA = 4 THEN 'PRIVADA COM FINS LUCRATIVOS'
                     WHEN TP_CATEGORIA_ADMINISTRATIVA = 5 THEN 'PRIVADA SEM FINS LUCRATIVOS'
                     WHEN TP_CATEGORIA_ADMINISTRATIVA = 6 THEN 'PRIVADA - PARTICULAR EM SENTIDO ESTRITO'
                     WHEN TP_CATEGORIA_ADMINISTRATIVA = 7 THEN 'ESPECIAL'
                     WHEN TP_CATEGORIA_ADMINISTRATIVA = 8 THEN 'PRIVADA COMUNITARIA'
                     WHEN TP_CATEGORIA_ADMINISTRATIVA = 9 THEN 'PRIVADA CONFESSIONAL'
                     ELSE 'N/A'
                END AS NM_CATEGORIA_ADMINISTRATIVA,
                COALESCE(TP_ORGANIZACAO_ACADEMICA,-3) AS TP_ORGANIZACAO_ACADEMICA,
                CASE WHEN TP_ORGANIZACAO_ACADEMICA = 1 THEN 'UNIVERSIDADE'
                     WHEN TP_ORGANIZACAO_ACADEMICA = 2 THEN 'CENTRO UNIVERSITARIO'
                     WHEN TP_ORGANIZACAO_ACADEMICA = 3 THEN 'FACULDADE'
                     WHEN TP_ORGANIZACAO_ACADEMICA = 4 THEN 'INSTITUTO FEDERAL DE EDUCACAO, CIENCIA E TECNOLOGIA'
                     WHEN TP_ORGANIZACAO_ACADEMICA = 5 THEN 'CENTRO FEDERAL DE EDUCACAO TECNOLOGICA'
                     ELSE 'N/A'
                END AS NM_ORGANIZACAO_ACADEMICA,
                COALESCE(CO_REGIAO,-3) AS CO_REGIAO,
                COALESCE(CO_UF,-3) AS CO_UF,
                COALESCE(CO_MUNICIPIO,-3) AS CO_MUNICIPIO,
                COALESCE(IN_CAPITAL,-3) AS IN_CAPITAL,
                COALESCE(QT_TEC_TOTAL,-3) AS QT_TEC_TOTAL,
                COALESCE(QT_TEC_FUNDAMENTAL_INCOMP_FEM,-3) AS QT_TEC_FUNDAMENTAL_INCOMP_FEM,
                COALESCE(QT_TEC_FUNDAMENTAL_INCOMP_MASC,-3) AS QT_TEC_FUNDAMENTAL_INCOMP_MASC,
                COALESCE(QT_TEC_FUNDAMENTAL_COMP_FEM,-3) AS QT_TEC_FUNDAMENTAL_COMP_FEM,
                COALESCE(QT_TEC_FUNDAMENTAL_COMP_MASC,-3) AS QT_TEC_FUNDAMENTAL_COMP_MASC,
                COALESCE(QT_TEC_MEDIO_FEM,-3) AS QT_TEC_MEDIO_FEM,
                COALESCE(QT_TEC_MEDIO_MASC,-3) AS QT_TEC_MEDIO_MASC,
                COALESCE(QT_TEC_SUPERIOR_FEM,-3) AS QT_TEC_SUPERIOR_FEM,
                COALESCE(QT_TEC_SUPERIOR_MASC,-3) AS QT_TEC_SUPERIOR_MASC,
                COALESCE(QT_TEC_ESPECIALIZACAO_FEM,-3) AS QT_TEC_ESPECIALIZACAO_FEM,
                COALESCE(QT_TEC_ESPECIALIZACAO_MASC,-3) AS QT_TEC_ESPECIALIZACAO_MASC,
                COALESCE(QT_TEC_MESTRADO_FEM,-3) AS QT_TEC_MESTRADO_FEM,
                COALESCE(QT_TEC_MESTRADO_MASC,-3) AS QT_TEC_MESTRADO_MASC,
                COALESCE(QT_TEC_DOUTORADO_FEM,-3) AS QT_TEC_DOUTORADO_FEM,
                COALESCE(QT_TEC_DOUTORADO_MASC,-3) AS QT_TEC_DOUTORADO_MASC,
                COALESCE(IN_ACESSO_PORTAL_CAPES,-3) AS IN_ACESSO_PORTAL_CAPES,
                COALESCE(IN_ACESSO_OUTRAS_BASES,-3) AS IN_ACESSO_OUTRAS_BASES,
                COALESCE(IN_ASSINA_OUTRA_BASE,-3) AS IN_ASSINA_OUTRA_BASE,
                COALESCE(IN_REPOSITORIO_INSTITUCIONAL,-3) AS IN_REPOSITORIO_INSTITUCIONAL,
                COALESCE(IN_BUSCA_INTEGRADA,-3) AS IN_BUSCA_INTEGRADA,
                COALESCE(IN_SERVICO_INTERNET,-3) AS IN_SERVICO_INTERNET,
                COALESCE(IN_PARTICIPA_REDE_SOCIAL,-3) AS IN_PARTICIPA_REDE_SOCIAL,
                COALESCE(IN_CATALOGO_ONLINE,-3) AS IN_CATALOGO_ONLINE,
                COALESCE(QT_PERIODICO_ELETRONICO,-3) AS QT_PERIODICO_ELETRONICO,
                COALESCE(QT_LIVRO_ELETRONICO,-3) AS QT_LIVRO_ELETRONICO,
                COALESCE(TP_REFERENTE,-3) AS TP_REFERENTE,
                CASE WHEN TP_REFERENTE = 1 THEN 'MANTENEDORA'
                    WHEN TP_REFERENTE = 2 THEN 'INSTITUICAO'
                ELSE 'N/A' END AS NM_REFERENTE,
                COALESCE(VL_RECEITA_PROPRIA,-3) AS VL_RECEITA_PROPRIA,
                COALESCE(VL_RECEITA_TRANSFERENCIA,-3) AS VL_RECEITA_TRANSFERENCIA,
                COALESCE(VL_RECEITA_OUTRA,-3) AS VL_RECEITA_OUTRA,
                COALESCE(VL_DESPESA_PESSOAL_DOCENTE,-3) AS VL_DESPESA_PESSOAL_DOCENTE,
                COALESCE(VL_DESPESA_PESSOAL_TECNICO,-3) AS VL_DESPESA_PESSOAL_TECNICO,
                COALESCE(VL_DESPESA_PESSOAL_ENCARGO,-3) AS VL_DESPESA_PESSOAL_ENCARGO,
                COALESCE(VL_DESPESA_CUSTEIO,-3) AS VL_DESPESA_CUSTEIO,
                COALESCE(VL_DESPESA_INVESTIMENTO,-3) AS VL_DESPESA_INVESTIMENTO,
                COALESCE(VL_DESPESA_PESQUISA,-3) AS VL_DESPESA_PESQUISA,
                COALESCE(VL_DESPESA_OUTRA,-3) AS VL_DESPESA_OUTRA
                FROM stg_ies""").createOrReplaceTempView('DIM_IES')
        
    ###LIMPEZA DE DADOS DOS ALUNOS
    spark.sql("""SELECT 
                COALESCE(SHA2(ID_ALUNO||CO_ALUNO_CURSO, 256),'N/A') AS CD_HASH,
                COALESCE(NU_ANO_CENSO, -3) AS NU_ANO_CENSO,
                COALESCE(CO_IES, -3) AS CO_IES,
                COALESCE(TP_CATEGORIA_ADMINISTRATIVA, -3) AS TP_CATEGORIA_ADMINISTRATIVA,
                COALESCE(TP_ORGANIZACAO_ACADEMICA, -3) AS TP_ORGANIZACAO_ACADEMICA,
                COALESCE(CO_CURSO, -3) AS CO_CURSO,
                COALESCE(CO_CURSO_POLO, -3) AS CO_CURSO_POLO,
                COALESCE(TP_TURNO, -3) AS TP_TURNO,
                CASE WHEN TP_TURNO = 1 THEN 'MATUTINO'
                     WHEN TP_TURNO = 2 THEN 'VESPERTINO'
                     WHEN TP_TURNO = 3 THEN 'NOTURNO'
                     WHEN TP_TURNO = 4 THEN 'INTEGRAL'
                ELSE 'N/A' END AS NM_TURNO,
                COALESCE(TP_GRAU_ACADEMICO, -3) AS TP_GRAU_ACADEMICO,
                COALESCE(TP_MODALIDADE_ENSINO, -3) AS TP_MODALIDADE_ENSINO,
                COALESCE(TP_NIVEL_ACADEMICO, -3) AS TP_NIVEL_ACADEMICO,
                COALESCE(CO_CINE_ROTULO, 'N/A') AS CO_CINE_ROTULO,
                COALESCE(ID_ALUNO, -3) AS ID_ALUNO,
                COALESCE(CO_ALUNO_CURSO, 'N/A') AS CO_ALUNO_CURSO,
                COALESCE(CO_ALUNO_CURSO_ORIGEM, -3) AS CO_ALUNO_CURSO_ORIGEM,
                COALESCE(TP_COR_RACA, -3) AS CD_COR_RACA,
                CASE WHEN TP_COR_RACA = 0 THEN 'NAO DECLARADO'
                     WHEN TP_COR_RACA = 1 THEN 'BRANCA'
                     WHEN TP_COR_RACA = 2 THEN 'PRETA'
                     WHEN TP_COR_RACA = 3 THEN 'PARDA'
                     WHEN TP_COR_RACA = 4 THEN 'AMARELA'
                     WHEN TP_COR_RACA = 5 THEN 'INDIGENA'
                     WHEN TP_COR_RACA = 9 THEN 'SEM RESPOSTA'
                ELSE 'N/A'
                END AS TP_COR_RACA,
                COALESCE(TP_SEXO, -3) AS TP_SEXO,
                CASE WHEN TP_SEXO = 1 THEN 'FEMININO'
                     WHEN TP_SEXO = 2 THEN 'MASCULINO'
                ELSE 'N/A' END AS NM_SEXO,
                COALESCE(NU_ANO_NASCIMENTO, -3) AS NU_ANO_NASCIMENTO,
                COALESCE(NU_MES_NASCIMENTO, -3) AS NU_MES_NASCIMENTO,
                COALESCE(NU_DIA_NASCIMENTO, -3) AS NU_DIA_NASCIMENTO,
                COALESCE(NU_IDADE, 0) AS NU_IDADE,
                COALESCE(TP_NACIONALIDADE, -3) AS TP_NACIONALIDADE,
                CASE WHEN TP_NACIONALIDADE = 1 THEN 'BRASILEIRA'
                     WHEN TP_NACIONALIDADE = 2 THEN 'BRASILEIRA - NATURALIZADO'
                     WHEN TP_NACIONALIDADE = 3 THEN 'ESTRANGEIRA'
                     ELSE 'N/A'
                END AS NM_NACIONALIDADE,
                COALESCE(CO_PAIS_ORIGEM, -3) AS CO_PAIS_ORIGEM,
                COALESCE(CO_UF_NASCIMENTO, -3) AS CO_UF_NASCIMENTO,
                COALESCE(CO_MUNICIPIO_NASCIMENTO, -3) AS CO_MUNICIPIO_NASCIMENTO,
                CASE WHEN IN_DEFICIENCIA = 9 THEN 0 
                     ELSE COALESCE(IN_DEFICIENCIA , 0) 
                END AS IN_DEFICIENCIA,
                COALESCE(IN_DEFICIENCIA_AUDITIVA, 0) AS IN_DEFICIENCIA_AUDITIVA,
                COALESCE(IN_DEFICIENCIA_FISICA, 0) AS IN_DEFICIENCIA_FISICA,
                COALESCE(IN_DEFICIENCIA_INTELECTUAL, 0) AS IN_DEFICIENCIA_INTELECTUAL,
                COALESCE(IN_DEFICIENCIA_MULTIPLA, 0) AS IN_DEFICIENCIA_MULTIPLA,
                COALESCE(IN_DEFICIENCIA_SURDEZ, 0) AS IN_DEFICIENCIA_SURDEZ,
                COALESCE(IN_DEFICIENCIA_SURDOCEGUEIRA, 0) AS IN_DEFICIENCIA_SURDOCEGUEIRA,
                COALESCE(IN_DEFICIENCIA_BAIXA_VISAO, 0) AS IN_DEFICIENCIA_BAIXA_VISAO,
                COALESCE(IN_DEFICIENCIA_CEGUEIRA, 0) AS IN_DEFICIENCIA_CEGUEIRA,
                COALESCE(IN_DEFICIENCIA_SUPERDOTACAO, 0) AS IN_DEFICIENCIA_SUPERDOTACAO,
                COALESCE(IN_TGD_AUTISMO, 0) AS IN_TGD_AUTISMO,
                COALESCE(IN_TGD_SINDROME_ASPERGER, 0) AS IN_TGD_SINDROME_ASPERGER,
                COALESCE(IN_TGD_SINDROME_RETT, 0) AS IN_TGD_SINDROME_RETT,
                COALESCE(IN_TGD_TRANSTOR_DESINTEGRATIVO, 0) AS IN_TGD_TRANSTOR_DESINTEGRATIVO,
                COALESCE(TP_SITUACAO, -3) AS TP_SITUACAO,
                CASE WHEN TP_SITUACAO = 2 THEN 'CURSANDO'
                     WHEN TP_SITUACAO = 3 THEN 'MATRICULA TRANCADA'
                     WHEN TP_SITUACAO = 4 THEN 'DESVINCULADO DO CURSO'
                     WHEN TP_SITUACAO = 5 THEN 'TRANSFERIDO PARA OUTRO CURSO DA IES'
                     WHEN TP_SITUACAO = 6 THEN 'FORMADO'
                     WHEN TP_SITUACAO = 7 THEN 'FALECIDO'
                     ELSE 'N/A' END AS NM_SITUACAO,
                COALESCE(QT_CARGA_HORARIA_TOTAL, -3) AS QT_CARGA_HORARIA_TOTAL,
                COALESCE(QT_CARGA_HORARIA_INTEG, -3) AS QT_CARGA_HORARIA_INTEG,
                COALESCE(DT_INGRESSO_CURSO, '1900-01-01') AS DT_INGRESSO_CURSO,
                COALESCE(IN_INGRESSO_VESTIBULAR, 0) AS IN_INGRESSO_VESTIBULAR,
                COALESCE(IN_INGRESSO_ENEM, 0) AS IN_INGRESSO_ENEM,
                COALESCE(IN_INGRESSO_AVALIACAO_SERIADA, 0) AS IN_INGRESSO_AVALIACAO_SERIADA,
                COALESCE(IN_INGRESSO_SELECAO_SIMPLIFICA, 0) AS IN_INGRESSO_SELECAO_SIMPLIFICA,
                COALESCE(IN_INGRESSO_OUTRO_TIPO_SELECAO, 0) AS IN_INGRESSO_OUTRO_TIPO_SELECAO,
                COALESCE(IN_INGRESSO_VAGA_REMANESC, 0) AS IN_INGRESSO_VAGA_REMANESC,
                COALESCE(IN_INGRESSO_VAGA_PROG_ESPECIAL, 0) AS IN_INGRESSO_VAGA_PROG_ESPECIAL,
                COALESCE(IN_INGRESSO_TRANSF_EXOFFICIO, 0) AS IN_INGRESSO_TRANSF_EXOFFICIO,
                COALESCE(IN_INGRESSO_DECISAO_JUDICIAL, 0) AS IN_INGRESSO_DECISAO_JUDICIAL,
                COALESCE(IN_INGRESSO_CONVENIO_PECG, 0) AS IN_INGRESSO_CONVENIO_PECG,
                COALESCE(IN_INGRESSO_EGRESSO, 0) AS IN_INGRESSO_EGRESSO,
                COALESCE(IN_INGRESSO_OUTRA_FORMA, 0) AS IN_INGRESSO_OUTRA_FORMA,
                COALESCE(IN_RESERVA_VAGAS, 0) AS IN_RESERVA_VAGAS,
                COALESCE(IN_RESERVA_ETNICO, 0) AS IN_RESERVA_ETNICO,
                COALESCE(IN_RESERVA_DEFICIENCIA, 0) AS IN_RESERVA_DEFICIENCIA,
                COALESCE(IN_RESERVA_ENSINO_PUBLICO, 0) AS IN_RESERVA_ENSINO_PUBLICO,
                COALESCE(IN_RESERVA_RENDA_FAMILIAR, 0) AS IN_RESERVA_RENDA_FAMILIAR,
                COALESCE(IN_RESERVA_OUTRA, 0) AS IN_RESERVA_OUTRA,
                COALESCE(IN_FINANCIAMENTO_ESTUDANTIL, 0) AS IN_FINANCIAMENTO_ESTUDANTIL,
                COALESCE(IN_FIN_REEMB_FIES, 0) AS IN_FIN_REEMB_FIES,
                COALESCE(IN_FIN_REEMB_ESTADUAL, 0) AS IN_FIN_REEMB_ESTADUAL,
                COALESCE(IN_FIN_REEMB_MUNICIPAL, 0) AS IN_FIN_REEMB_MUNICIPAL,
                COALESCE(IN_FIN_REEMB_PROG_IES, 0) AS IN_FIN_REEMB_PROG_IES,
                COALESCE(IN_FIN_REEMB_ENT_EXTERNA, 0) AS IN_FIN_REEMB_ENT_EXTERNA,
                COALESCE(IN_FIN_REEMB_OUTRA, 0) AS IN_FIN_REEMB_OUTRA,
                COALESCE(IN_FIN_NAOREEMB_PROUNI_INTEGR, 0) AS IN_FIN_NAOREEMB_PROUNI_INTEGR,
                COALESCE(IN_FIN_NAOREEMB_PROUNI_PARCIAL, 0) AS IN_FIN_NAOREEMB_PROUNI_PARCIAL,
                COALESCE(IN_FIN_NAOREEMB_ESTADUAL, 0) AS IN_FIN_NAOREEMB_ESTADUAL,
                COALESCE(IN_FIN_NAOREEMB_MUNICIPAL, 0) AS IN_FIN_NAOREEMB_MUNICIPAL,
                COALESCE(IN_FIN_NAOREEMB_PROG_IES, 0) AS IN_FIN_NAOREEMB_PROG_IES,
                COALESCE(IN_FIN_NAOREEMB_ENT_EXTERNA, 0) AS IN_FIN_NAOREEMB_ENT_EXTERNA,
                COALESCE(IN_FIN_NAOREEMB_OUTRA, 0) AS IN_FIN_NAOREEMB_OUTRA,
                COALESCE(IN_APOIO_SOCIAL, 0) AS IN_APOIO_SOCIAL,
                COALESCE(IN_APOIO_ALIMENTACAO, 0) AS IN_APOIO_ALIMENTACAO,
                COALESCE(IN_APOIO_BOLSA_PERMANENCIA, 0) AS IN_APOIO_BOLSA_PERMANENCIA,
                COALESCE(IN_APOIO_BOLSA_TRABALHO, 0) AS IN_APOIO_BOLSA_TRABALHO,
                COALESCE(IN_APOIO_MATERIAL_DIDATICO, 0) AS IN_APOIO_MATERIAL_DIDATICO,
                COALESCE(IN_APOIO_MORADIA, 0) AS IN_APOIO_MORADIA,
                COALESCE(IN_APOIO_TRANSPORTE, 0) AS IN_APOIO_TRANSPORTE,
                COALESCE(IN_ATIVIDADE_EXTRACURRICULAR, 0) AS IN_ATIVIDADE_EXTRACURRICULAR,
                COALESCE(IN_COMPLEMENTAR_ESTAGIO, 0) AS IN_COMPLEMENTAR_ESTAGIO,
                COALESCE(IN_COMPLEMENTAR_EXTENSAO, 0) AS IN_COMPLEMENTAR_EXTENSAO,
                COALESCE(IN_COMPLEMENTAR_MONITORIA, 0) AS IN_COMPLEMENTAR_MONITORIA,
                COALESCE(IN_COMPLEMENTAR_PESQUISA, 0) AS IN_COMPLEMENTAR_PESQUISA,
                COALESCE(IN_BOLSA_ESTAGIO, 0) AS IN_BOLSA_ESTAGIO,
                COALESCE(IN_BOLSA_EXTENSAO, 0) AS IN_BOLSA_EXTENSAO,
                COALESCE(IN_BOLSA_MONITORIA, 0) AS IN_BOLSA_MONITORIA,
                COALESCE(IN_BOLSA_PESQUISA, 0) AS IN_BOLSA_PESQUISA,
                COALESCE(TP_ESCOLA_CONCLUSAO_ENS_MEDIO, 0) AS TP_ESCOLA_CONCLUSAO_ENS_MEDIO,
                CASE WHEN TP_ESCOLA_CONCLUSAO_ENS_MEDIO = 1 THEN 'PUBLICA'
                     WHEN TP_ESCOLA_CONCLUSAO_ENS_MEDIO = 2 THEN 'PRIVADA'
                     WHEN TP_ESCOLA_CONCLUSAO_ENS_MEDIO = 9 THEN 'SEM INFO'
                ELSE 'N/A' END AS NM_ESCOLA_CONCLUSAO_ENS_MEDIO,

                COALESCE(IN_ALUNO_PARFOR, 0) AS IN_ALUNO_PARFOR,
                COALESCE(TP_SEMESTRE_CONCLUSAO, 0) AS TP_SEMESTRE_CONCLUSAO,
                COALESCE(TP_SEMESTRE_REFERENCIA, -3) AS TP_SEMESTRE_REFERENCIA,
                COALESCE(IN_MOBILIDADE_ACADEMICA, 0) AS IN_MOBILIDADE_ACADEMICA,
                COALESCE(TP_MOBILIDADE_ACADEMICA, -3) AS TP_MOBILIDADE_ACADEMICA,
                CASE WHEN TP_MOBILIDADE_ACADEMICA = 1 THEN 'NACIONAL'
                     WHEN TP_MOBILIDADE_ACADEMICA = 2 THEN 'INTERNACIONAL'
                     ELSE 'N/A' 
                END AS NM_MOBILIDADE_ACADEMICA,
                COALESCE(TP_MOBILIDADE_ACADEMICA_INTERN, -3) AS TP_MOBILIDADE_ACADEMICA_INTERN,
                CASE WHEN TP_MOBILIDADE_ACADEMICA_INTERN = 1 THEN 'INTERCAMBIO' 
                     WHEN TP_MOBILIDADE_ACADEMICA_INTERN = 2 THEN 'CIENCIA SEM FRONTEIRAS'
                     ELSE 'N/A' 
                END NM_MOBILIDADE_ACADEMICA_INTER,
                COALESCE(CO_IES_DESTINO, -3) AS CO_IES_DESTINO,
                COALESCE(CO_PAIS_DESTINO, -3) AS CO_PAIS_DESTINO,
                COALESCE(IN_MATRICULA, 0) AS IN_MATRICULA,
                COALESCE(IN_CONCLUINTE, 0) AS IN_CONCLUINTE,
                COALESCE(IN_INGRESSO_TOTAL, 0) AS IN_INGRESSO_TOTAL,
                COALESCE(IN_INGRESSO_VAGA_NOVA, 0) AS IN_INGRESSO_VAGA_NOVA,
                COALESCE(IN_INGRESSO_PROCESSO_SELETIVO, 0) AS IN_INGRESSO_PROCESSO_SELETIVO,
                COALESCE(NU_ANO_INGRESSO, -3) AS NU_ANO_INGRESSO
                from stg_aluno""").createOrReplaceTempView('DIM_ALUNO')
    ### limpeza dados de cursos
    spark.sql("""SELECT COALESCE(NU_ANO_CENSO,-3) AS NU_ANO_CENSO,
                    COALESCE(CO_IES,-3) AS CO_IES,
                    COALESCE(TP_CATEGORIA_ADMINISTRATIVA,-3) AS TP_CATEGORIA_ADMINISTRATIVA,
                    COALESCE(TP_ORGANIZACAO_ACADEMICA,-3) AS TP_ORGANIZACAO_ACADEMICA,
                    COALESCE(CO_LOCAL_OFERTA,-3) AS CO_LOCAL_OFERTA,
                    COALESCE(CO_UF,-3) AS CO_UF,
                    COALESCE(CO_MUNICIPIO,-3) AS CO_MUNICIPIO,
                    COALESCE(IN_CAPITAL,-3) AS IN_CAPITAL,
                    COALESCE(CO_CURSO,-3) AS CO_CURSO,
                    COALESCE(NO_CURSO,'N/A') AS NO_CURSO,
                    CASE WHEN TP_SITUACAO = 1 THEN 'EM ATIVIDADE'
                        WHEN TP_SITUACAO = 2 THEN 'EXTINTO'
                        WHEN TP_SITUACAO = 3 THEN 'EM EXTINCAO'
                    ELSE 'N/A' END AS TP_SITUACAO,
                    COALESCE(CO_CINE_ROTULO,'N/A') AS CO_CINE_ROTULO,
                    CASE WHEN TP_GRAU_ACADEMICO = 1 THEN 'BACHARELADO'
                        WHEN TP_GRAU_ACADEMICO = 2 THEN 'LICENCIATURA'
                        WHEN TP_GRAU_ACADEMICO = 3 THEN 'TECNOLOGICO'
                        WHEN TP_GRAU_ACADEMICO = 4 THEN 'BACHARELADO E LICENCIATURA'
                    ELSE 'N/A' END AS TP_GRAU_ACADEMICO,
                    CASE WHEN TP_MODALIDADE_ENSINO = 1 THEN 'PRESENCIAL'
                        WHEN TP_MODALIDADE_ENSINO = 2 THEN 'CURSO A DISTANCIA'
                    ELSE 'N/A' END TP_MODALIDADE_ENSINO,
                    CASE WHEN TP_NIVEL_ACADEMICO = 1 THEN 'GRADUACAO'
                         WHEN TP_NIVEL_ACADEMICO = 2 THEN 'SEQUENCIAL DE FORMACAO ESPECIFICA'
                         ELSE 'N/A' 
                    END AS TP_NIVEL_ACADEMICO,
                    COALESCE(IN_GRATUITO,-3) AS IN_GRATUITO,
                    CASE WHEN TP_ATRIBUTO_INGRESSO = 0 THEN 'NORMAL'
                        WHEN TP_ATRIBUTO_INGRESSO = 1 THEN 'AREA BASICA DE INGRESSO'
                        WHEN TP_ATRIBUTO_INGRESSO = 2 THEN 'BRACHARELADO OU LICENCIATURA INTERDISCIPLINAR'
                        ELSE 'N/A'
                    END AS TP_ATRIBUTO_INGRESSO,
                    COALESCE(NU_CARGA_HORARIA,-3) AS NU_CARGA_HORARIA,
                    COALESCE(DT_INICIO_FUNCIONAMENTO,'1900-01-01') AS DT_INICIO_FUNCIONAMENTO,
                    COALESCE(DT_AUTORIZACAO_CURSO,'1900-01-01') AS DT_AUTORIZACAO_CURSO,
                    COALESCE(IN_AJUDA_DEFICIENTE,-3) AS IN_AJUDA_DEFICIENTE,
                    COALESCE(IN_MATERIAL_DIGITAL,-3) AS IN_MATERIAL_DIGITAL,
                    COALESCE(IN_MATERIAL_AMPLIADO,-3) AS IN_MATERIAL_AMPLIADO,
                    COALESCE(IN_MATERIAL_TATIL,-3) AS IN_MATERIAL_TATIL,
                    COALESCE(IN_MATERIAL_IMPRESSO,-3) AS IN_MATERIAL_IMPRESSO,
                    COALESCE(IN_MATERIAL_AUDIO,-3) AS IN_MATERIAL_AUDIO,
                    COALESCE(IN_MATERIAL_BRAILLE,-3) AS IN_MATERIAL_BRAILLE,
                    COALESCE(IN_MATERIAL_LIBRAS,-3) AS IN_MATERIAL_LIBRAS,
                    COALESCE(IN_DISCIPLINA_LIBRAS,-3) AS IN_DISCIPLINA_LIBRAS,
                    COALESCE(IN_TRADUTOR_LIBRAS,-3) AS IN_TRADUTOR_LIBRAS,
                    COALESCE(IN_GUIA_INTERPRETE,-3) AS IN_GUIA_INTERPRETE,
                    COALESCE(IN_RECURSOS_COMUNICACAO,-3) AS IN_RECURSOS_COMUNICACAO,
                    COALESCE(IN_RECURSOS_INFORMATICA,-3) AS IN_RECURSOS_INFORMATICA,
                    COALESCE(IN_INTEGRAL,-3) AS IN_INTEGRAL,
                    COALESCE(IN_MATUTINO,-3) AS IN_MATUTINO,
                    COALESCE(IN_VESPERTINO,-3) AS IN_VESPERTINO,
                    COALESCE(IN_NOTURNO,-3) AS IN_NOTURNO,
                    COALESCE(NU_INTEGRALIZACAO_INTEGRAL,-3) AS NU_INTEGRALIZACAO_INTEGRAL,
                    COALESCE(NU_INTEGRALIZACAO_MATUTINO,-3) AS NU_INTEGRALIZACAO_MATUTINO,
                    COALESCE(NU_INTEGRALIZACAO_VESPERTINO,-3) AS NU_INTEGRALIZACAO_VESPERTINO,
                    COALESCE(NU_INTEGRALIZACAO_NOTURNO,-3) AS NU_INTEGRALIZACAO_NOTURNO,
                    COALESCE(NU_INTEGRALIZACAO_EAD,-3) AS NU_INTEGRALIZACAO_EAD,
                    COALESCE(IN_OFERECE_DISC_SEMI_PRES,-3) AS IN_OFERECE_DISC_SEMI_PRES,
                    COALESCE(NU_PERC_CARGA_SEMI_PRES,-3) AS NU_PERC_CARGA_SEMI_PRES,
                    COALESCE(IN_POSSUI_LABORATORIO,-3) AS IN_POSSUI_LABORATORIO,
                    COALESCE(QT_INSC_VAGA_NOVA_INTEGRAL,-3) AS QT_INSC_VAGA_NOVA_INTEGRAL,
                    COALESCE(QT_INSC_VAGA_NOVA_MATUTINO,-3) AS QT_INSC_VAGA_NOVA_MATUTINO,
                    COALESCE(QT_INSC_VAGA_NOVA_VESPERTINO,-3) AS QT_INSC_VAGA_NOVA_VESPERTINO,
                    COALESCE(QT_INSC_VAGA_NOVA_NOTURNO,-3) AS QT_INSC_VAGA_NOVA_NOTURNO,
                    COALESCE(QT_INSC_VAGA_NOVA_EAD,-3) AS QT_INSC_VAGA_NOVA_EAD,
                    COALESCE(QT_INSC_VAGA_REMAN_INTEGRAL,-3) AS QT_INSC_VAGA_REMAN_INTEGRAL,
                    COALESCE(QT_INSC_VAGA_REMAN_MATUTINO,-3) AS QT_INSC_VAGA_REMAN_MATUTINO,
                    COALESCE(QT_INSC_VAGA_REMAN_VESPERTINO,-3) AS QT_INSC_VAGA_REMAN_VESPERTINO,
                    COALESCE(QT_INSC_VAGA_REMAN_NOTURNO,-3) AS QT_INSC_VAGA_REMAN_NOTURNO,
                    COALESCE(QT_INSC_VAGA_REMAN_EAD,-3) AS QT_INSC_VAGA_REMAN_EAD,
                    COALESCE(QT_INSC_PROG_ESP_INTEGRAL,-3) AS QT_INSC_PROG_ESP_INTEGRAL,
                    COALESCE(QT_INSC_PROG_ESP_MATUTINO,-3) AS QT_INSC_PROG_ESP_MATUTINO,
                    COALESCE(QT_INSC_PROG_ESP_VESPERTINO,-3) AS QT_INSC_PROG_ESP_VESPERTINO,
                    COALESCE(QT_INSC_PROG_ESP_NOTURNO,-3) AS QT_INSC_PROG_ESP_NOTURNO,
                    COALESCE(QT_INSC_PROG_ESP_EAD,-3) AS QT_INSC_PROG_ESP_EAD,
                    COALESCE(QT_INSC_PRINCIPAL_INTEGRAL,-3) AS QT_INSC_PRINCIPAL_INTEGRAL,
                    COALESCE(QT_INSC_PRINCIPAL_MATUTINO,-3) AS QT_INSC_PRINCIPAL_MATUTINO,
                    COALESCE(QT_INSC_PRINCIPAL_VESPERTINO,-3) AS QT_INSC_PRINCIPAL_VESPERTINO,
                    COALESCE(QT_INSC_PRINCIPAL_NOTURNO,-3) AS QT_INSC_PRINCIPAL_NOTURNO,
                    COALESCE(QT_INSC_PRINCIPAL_EAD,-3) AS QT_INSC_PRINCIPAL_EAD,
                    COALESCE(QT_INSC_OUTRA_VAGA_INTEGRAL,-3) AS QT_INSC_OUTRA_VAGA_INTEGRAL,
                    COALESCE(QT_INSC_OUTRA_VAGA_MATUTINO,-3) AS QT_INSC_OUTRA_VAGA_MATUTINO,
                    COALESCE(QT_INSC_OUTRA_VAGA_VESPERTINO,-3) AS QT_INSC_OUTRA_VAGA_VESPERTINO,
                    COALESCE(QT_INSC_OUTRA_VAGA_NOTURNO,-3) AS QT_INSC_OUTRA_VAGA_NOTURNO,
                    COALESCE(QT_INSC_OUTRA_VAGA_EAD,-3) AS QT_INSC_OUTRA_VAGA_EAD,
                    COALESCE(QT_INSC_ANUAL_INTEGRAL,-3) AS QT_INSC_ANUAL_INTEGRAL,
                    COALESCE(QT_INSC_ANUAL_MATUTINO,-3) AS QT_INSC_ANUAL_MATUTINO,
                    COALESCE(QT_INSC_ANUAL_VESPERTINO,-3) AS QT_INSC_ANUAL_VESPERTINO,
                    COALESCE(QT_INSC_ANUAL_NOTURNO,-3) AS QT_INSC_ANUAL_NOTURNO,
                    COALESCE(QT_INSC_ANUAL_EAD,-3) AS QT_INSC_ANUAL_EAD,
                    COALESCE(QT_VAGAS_NOVAS_INTEGRAL,-3) AS QT_VAGAS_NOVAS_INTEGRAL,
                    COALESCE(QT_VAGAS_NOVAS_MATUTINO,-3) AS QT_VAGAS_NOVAS_MATUTINO,
                    COALESCE(QT_VAGAS_NOVAS_VESPERTINO,-3) AS QT_VAGAS_NOVAS_VESPERTINO,
                    COALESCE(QT_VAGAS_NOVAS_NOTURNO,-3) AS QT_VAGAS_NOVAS_NOTURNO,
                    COALESCE(QT_VAGAS_NOVAS_EAD,-3) AS QT_VAGAS_NOVAS_EAD,
                    COALESCE(QT_VAGAS_REMAN_INTEGRAL,-3) AS QT_VAGAS_REMAN_INTEGRAL,
                    COALESCE(QT_VAGAS_REMAN_MATUTINO,-3) AS QT_VAGAS_REMAN_MATUTINO,
                    COALESCE(QT_VAGAS_REMAN_VESPERTINO,-3) AS QT_VAGAS_REMAN_VESPERTINO,
                    COALESCE(QT_VAGAS_REMAN_NOTURNO,-3) AS QT_VAGAS_REMAN_NOTURNO,
                    COALESCE(QT_VAGAS_REMAN_EAD,-3) AS QT_VAGAS_REMAN_EAD,
                    COALESCE(QT_VAGAS_PROG_ESP_INTEGRAL,-3) AS QT_VAGAS_PROG_ESP_INTEGRAL,
                    COALESCE(QT_VAGAS_PROG_ESP_MATUTINO,-3) AS QT_VAGAS_PROG_ESP_MATUTINO,
                    COALESCE(QT_VAGAS_PROG_ESP_VESPERTINO,-3) AS QT_VAGAS_PROG_ESP_VESPERTINO,
                    COALESCE(QT_VAGAS_PROG_ESP_NOTURNO,-3) AS QT_VAGAS_PROG_ESP_NOTURNO,
                    COALESCE(QT_VAGAS_PROG_ESP_EAD,-3) AS QT_VAGAS_PROG_ESP_EAD,
                    COALESCE(QT_VAGAS_PRINCIPAL_INTEGRAL,-3) AS QT_VAGAS_PRINCIPAL_INTEGRAL,
                    COALESCE(QT_VAGAS_PRINCIPAL_MATUTINO,-3) AS QT_VAGAS_PRINCIPAL_MATUTINO,
                    COALESCE(QT_VAGAS_PRINCIPAL_VESPERTINO,-3) AS QT_VAGAS_PRINCIPAL_VESPERTINO,
                    COALESCE(QT_VAGAS_PRINCIPAL_NOTURNO,-3) AS QT_VAGAS_PRINCIPAL_NOTURNO,
                    COALESCE(QT_VAGAS_PRINCIPAL_EAD,-3) AS QT_VAGAS_PRINCIPAL_EAD,
                    COALESCE(QT_VAGAS_OUTRAS_INTEGRAL,-3) AS QT_VAGAS_OUTRAS_INTEGRAL,
                    COALESCE(QT_VAGAS_OUTRAS_MATUTINO,-3) AS QT_VAGAS_OUTRAS_MATUTINO,
                    COALESCE(QT_VAGAS_OUTRAS_VESPERTINO,-3) AS QT_VAGAS_OUTRAS_VESPERTINO,
                    COALESCE(QT_VAGAS_OUTRAS_NOTURNO,-3) AS QT_VAGAS_OUTRAS_NOTURNO,
                    COALESCE(QT_VAGAS_OUTRAS_EAD,-3) AS QT_VAGAS_OUTRAS_EAD,
                    COALESCE(QT_VAGAS_ANUAL_INTEGRAL,-3) AS QT_VAGAS_ANUAL_INTEGRAL,
                    COALESCE(QT_VAGAS_ANUAL_MATUTINO,-3) AS QT_VAGAS_ANUAL_MATUTINO,
                    COALESCE(QT_VAGAS_ANUAL_VESPERTINO,-3) AS QT_VAGAS_ANUAL_VESPERTINO,
                    COALESCE(QT_VAGAS_ANUAL_NOTURNO,-3) AS QT_VAGAS_ANUAL_NOTURNO,
                    COALESCE(QT_VAGAS_ANUAL_EAD,-3) AS QT_VAGAS_ANUAL_EAD,
                    COALESCE(QT_MATRICULA_TOTAL,-3) AS QT_MATRICULA_TOTAL,
                    COALESCE(QT_CONCLUINTE_TOTAL,-3) AS QT_CONCLUINTE_TOTAL,
                    COALESCE(QT_INGRESSO_TOTAL,-3) AS QT_INGRESSO_TOTAL,
                    COALESCE(QT_INGRESSO_VAGA_NOVA,-3) AS QT_INGRESSO_VAGA_NOVA,
                    COALESCE(QT_INGRESSO_PROCESSO_SELETIVO,-3) AS QT_INGRESSO_PROCESSO_SELETIVO,
                    COALESCE(QT_VAGA_TOTAL,-3) AS QT_VAGA_TOTAL,
                    COALESCE(QT_INSCRITO_TOTAL, -3) AS QT_INSCRITO_TOTAL
                    FROM stg_curso""").createOrReplaceTempView('DIM_CURSO')

In [ ]:
write_file_parquet(read_file_csv('DM_CURSO.CSV'),'stg', 'DM_CURSO')

In [ ]:
write_file_parquet(read_file_csv('DM_ALUNOS.CSV'),'stg','DM_ALUNO')

In [ ]:
write_file_parquet(read_file_csv('DM_IES.CSV'),'stg','DM_IES')

In [ ]:
write_file_parquet(read_file_csv('municipios.csv','utf8'),'stg','Municipio')

In [ ]:
read_file_parquet('DM_CURSO').createOrReplaceTempView("stg_curso")

In [ ]:
read_file_parquet('DM_ALUNO').createOrReplaceTempView("stg_aluno")

In [ ]:
read_file_parquet('DM_IES').createOrReplaceTempView("stg_ies")

In [ ]:
read_file_parquet('Municipio').createOrReplaceTempView("stg_municipio_ibge")

In [ ]:
clean_data()

In [ ]:
write_bigquery(spark.sql("select * from DIM_CURSO"),'DIM_CURSO')

In [ ]:
write_bigquery(spark.sql("select * from DIM_LOCALIZACAO"),'DIM_LOCALIZACAO')

In [ ]:
write_bigquery(spark.sql("select * from DIM_IES"),'DIM_IES')

In [ ]:
write_bigquery(spark.sql("""SELECT
                    ALU.NM_SEXO,
                    ALU.TP_COR_RACA,
                    ALU.NM_TURNO,
                    ALU.NU_IDADE,
                    ALU.NM_SITUACAO,
                    ALU.TP_GRAU_ACADEMICO,
                    ALU.NM_ESCOLA_CONCLUSAO_ENS_MEDIO,
                    cur.CO_CURSO,
                    ies.CO_IES,
                    loc.cd_uf,
                    loc.CD_MUNICIPIO_COMPLETO,
                    sha2(loc.CD_UF||CD_MUNICIPIO_COMPLETO,256) as HASH_LOCALIZACAO,
                    ALU.NM_NACIONALIDADE,
                    COUNT(alu.CD_HASH) AS QT_ALUNOS,
                    SUM(alu.IN_DEFICIENCIA) as QT_ALUNOS_DEFICIENCIA,
                    SUM(alu.IN_DEFICIENCIA_AUDITIVA) as QT_ALUNOS_DEFICIENCIA_AUDITIVA,
                    SUM(alu.IN_DEFICIENCIA_FISICA) as QT_ALUNOS_DEFICIENCIA_FISICA,
                    SUM(alu.IN_DEFICIENCIA_INTELECTUAL) as QT_ALUNOS_DEFICIENCIA_INTELECTUAL,
                    SUM(alu.IN_DEFICIENCIA_MULTIPLA) as QT_ALUNOS_DEFICIENCIA_MULTIPLA,
                    SUM(alu.IN_DEFICIENCIA_SURDEZ) as QT_ALUNOS_DEFICIENCIA_SURDEZ,
                    SUM(alu.IN_DEFICIENCIA_SURDOCEGUEIRA) as QT_ALUNOS_DEFICIENCIA_SURDOCEGUEIRA,
                    SUM(alu.IN_DEFICIENCIA_BAIXA_VISAO) as QT_ALUNOS_DEFICIENCIA_BAIXA_VISAO,
                    SUM(alu.IN_DEFICIENCIA_CEGUEIRA) as QT_ALUNOS_DEFICIENCIA_CEGUEIRA,
                    SUM(alu.IN_TGD_AUTISMO) as QT_ALUNOS_TGD_AUTISMO,
                    SUM(alu.IN_TGD_SINDROME_ASPERGER) as QT_ALUNOS_TGD_SINDROME_ASPERGER,
                    SUM(alu.IN_TGD_TRANSTOR_DESINTEGRATIVO) as QT_ALUNOS_TGD_TRANSTOR_DESINTEGRATIVO,
                    SUM(alu.IN_INGRESSO_VESTIBULAR) as QT_ALUNOS_INGRESSO_VESTIBULAR,
                    SUM(alu.IN_INGRESSO_ENEM) as QT_ALUNOS_INGRESSO_ENEM,
                    SUM(alu.IN_INGRESSO_AVALIACAO_SERIADA) as QT_ALUNOS_INGRESSO_AVALIACAO_SERIADA,
                    SUM(alu.IN_INGRESSO_VAGA_REMANESC) as QT_ALUNOS_INGRESSO_VAGA_REMANESC,
                    SUM(alu.IN_INGRESSO_VAGA_PROG_ESPECIAL) as QT_ALUNOS_INGRESSO_VAGA_PROG_ESPECIAL,
                    SUM(alu.IN_INGRESSO_TRANSF_EXOFFICIO) as QT_ALUNOS_INGRESSO_TRANSF_EXOFFICIO,
                    SUM(alu.IN_INGRESSO_DECISAO_JUDICIAL) as QT_ALUNOS_INGRESSO_DECISAO_JUDICIAL,
                    SUM(alu.IN_RESERVA_VAGAS) as QT_ALUNOS_RESERVA_VAGAS,
                    SUM(alu.IN_RESERVA_ETNICO) as QT_ALUNOS_RESERVA_ETNICO,
                    SUM(alu.IN_RESERVA_DEFICIENCIA) as QT_ALUNOS_RESERVA_DEFICIENCIA,
                    SUM(alu.IN_RESERVA_ENSINO_PUBLICO) as QT_ALUNOS_RESERVA_ENSINO_PUBLICO,
                    SUM(alu.IN_RESERVA_RENDA_FAMILIAR) as QT_ALUNOS_RESERVA_RENDA_FAMILIAR,
                    SUM(alu.IN_RESERVA_OUTRA) as QT_ALUNOS_RESERVA_OUTRA,
                    SUM(alu.IN_FINANCIAMENTO_ESTUDANTIL) as QT_ALUNOS_FINANCIAMENTO_ESTUDANTIL,
                    SUM(alu.IN_FIN_REEMB_FIES) as QT_ALUNOS_FIN_REEMB_FIES,
                    SUM(alu.IN_FIN_REEMB_ESTADUAL) as QT_ALUNOS_FIN_REEMB_ESTADUAL,
                    SUM(alu.IN_FIN_REEMB_MUNICIPAL) as QT_ALUNOS_FIN_REEMB_MUNICIPAL,
                    SUM(alu.IN_FIN_REEMB_PROG_IES) as QT_ALUNOS_FIN_REEMB_PROG_IES,
                    SUM(alu.IN_FIN_REEMB_ENT_EXTERNA) as QT_ALUNOS_FIN_REEMB_ENT_EXTERNA,
                    SUM(alu.IN_FIN_REEMB_OUTRA) as QT_ALUNOS_FIN_REEMB_OUTRA,
                    SUM(alu.IN_FIN_NAOREEMB_PROUNI_INTEGR) as QT_ALUNOS_FIN_NAOREEMB_PROUNI_INTEGR,
                    SUM(alu.IN_FIN_NAOREEMB_PROUNI_PARCIAL) as QT_ALUNOS_FIN_NAOREEMB_PROUNI_PARCIAL,
                    SUM(alu.IN_FIN_NAOREEMB_ESTADUAL) as QT_ALUNOS_FIN_NAOREEMB_ESTADUAL,
                    SUM(alu.IN_FIN_NAOREEMB_MUNICIPAL) as QT_ALUNOS_FIN_NAOREEMB_MUNICIPAL,
                    SUM(alu.IN_FIN_NAOREEMB_PROG_IES) as QT_ALUNOS_FIN_NAOREEMB_PROG_IES,
                    SUM(alu.IN_FIN_NAOREEMB_ENT_EXTERNA) as QT_ALUNOS_FIN_NAOREEMB_ENT_EXTERNA,
                    SUM(alu.IN_FIN_NAOREEMB_OUTRA) as QT_ALUNOS_FIN_NAOREEMB_OUTRA,
                    SUM(alu.IN_APOIO_SOCIAL) as QT_ALUNOS_APOIO_SOCIAL,
                    SUM(alu.IN_APOIO_ALIMENTACAO) as QT_ALUNOS_APOIO_ALIMENTACAO,
                    SUM(alu.IN_APOIO_BOLSA_PERMANENCIA) as QT_ALUNOS_APOIO_BOLSA_PERMANENCIA,
                    SUM(alu.IN_APOIO_BOLSA_TRABALHO) as QT_ALUNOS_APOIO_BOLSA_TRABALHO,
                    SUM(alu.IN_APOIO_MATERIAL_DIDATICO) as QT_ALUNOS_APOIO_MATERIAL_DIDATICO,
                    SUM(alu.IN_APOIO_MORADIA) as QT_ALUNOS_APOIO_MORADIA,
                    SUM(alu.IN_APOIO_TRANSPORTE) as QT_ALUNOS_APOIO_TRANSPORTE,
                    SUM(alu.IN_ATIVIDADE_EXTRACURRICULAR) as QT_ALUNOS_ATIVIDADE_EXTRACURRICULAR,
                    SUM(alu.IN_COMPLEMENTAR_ESTAGIO) as QT_ALUNOS_COMPLEMENTAR_ESTAGIO,
                    SUM(alu.IN_COMPLEMENTAR_EXTENSAO) as QT_ALUNOS_COMPLEMENTAR_EXTENSAO,
                    SUM(alu.IN_COMPLEMENTAR_MONITORIA) as QT_ALUNOS_COMPLEMENTAR_MONITORIA,
                    SUM(alu.IN_COMPLEMENTAR_PESQUISA) as QT_ALUNOS_COMPLEMENTAR_PESQUISA,
                    SUM(alu.IN_BOLSA_ESTAGIO) as QT_ALUNOS_BOLSA_ESTAGIO,
                    SUM(alu.IN_BOLSA_EXTENSAO) as QT_ALUNOS_BOLSA_EXTENSAO,
                    SUM(alu.IN_BOLSA_MONITORIA) as QT_ALUNOS_BOLSA_MONITORIA,
                    SUM(alu.IN_BOLSA_PESQUISA) as QT_ALUNOS_BOLSA_PESQUISA,
                    SUM(alu.IN_MATRICULA) as QT_ALUNOS_MATRICULADOS,
                    SUM(alu.IN_CONCLUINTE) as QT_ALUNOS_CONCLUINTES
             FROM DIM_ALUNO alu 
                  INNER JOIN DIM_IES ies on(alu.CO_IES = IES.CO_IES)
                  INNER JOIN DIM_CURSO cur on(cur.CO_CURSO = alu.CO_CURSO)
                  INNER JOIN DIM_LOCALIZACAO loc on(loc.CD_UF = ies.CO_UF AND CD_MUNICIPIO_COMPLETO = ies.CO_MUNICIPIO)
                  GROUP BY NM_SEXO,
                           loc.cd_uf, 
                           cur.CO_CURSO,
                           ALU.TP_COR_RACA, 
                           ies.CO_IES, 
                           ALU.TP_GRAU_ACADEMICO,
                           ALU.NM_ESCOLA_CONCLUSAO_ENS_MEDIO,
                           ALU.NM_NACIONALIDADE,
                           ALU.NM_TURNO,
                           ALU.NU_IDADE,
                           ALU.NM_SITUACAO,
                           loc.CD_MUNICIPIO_COMPLETO,
                           sha2(loc.CD_UF||CD_MUNICIPIO_COMPLETO,256)
                           """), "FAT_CENSO_2019")

In [ ]:
!hadoop distcp hdfs://cluster-censo-educacional-m:8020//stg//parquet/DM_ALUNO/*.parquet gs://bkt-prd-portfolio/curated/censo/2018/DM_ALUNO/
!hadoop distcp hdfs://cluster-censo-educacional-m:8020//stg//parquet/DM_CURSO/*.parquet gs://bkt-prd-portfolio/curated/censo/2018/DM_CURSO/
!hadoop distcp hdfs://cluster-censo-educacional-m:8020//stg//parquet/DM_IES/*.parquet gs://bkt-prd-portfolio/curated/censo/2018/DM_IES/
!hadoop distcp hdfs://cluster-censo-educacional-m:8020//stg//parquet/Municipio/*.parquet gs://bkt-prd-portfolio/curated/censo/2018/Municipio/